In [14]:
import requests
import os
from dotenv import load_dotenv

from neo4j import GraphDatabase, RoutingControl

In [31]:
load_dotenv()

payload = {
    'api_key': os.getenv('CONGRESS_API'),
    'limit': 250
}

neo_driver = GraphDatabase.driver(os.getenv('NEO_URL'), auth=('',''))

In [16]:
def create_member(name):

    neo_driver.execute_query(
        'CREATE (a:Member {head: $name})',
        name=name, database_='neo4j'
    )

def add_party_member(member, party):

    neo_driver.execute_query(
        'MERGE (m: Member {head: $member}) '
        'MERGE (p: Party {head: $party}) '
        'MERGE (m)-[:MEMBER_OF]->(p)',
        member=member, party=party, database_='neo4j'
    )

def add_representation(member, state, district = None):

    if district is None:

        neo_driver.execute_query(
            'MERGE (m: Member {head: $member}) '
            'MERGE (s: State {head: $state}) '
            'MERGE (m)-[:REPRESENTS]->(s)',
            member=member, state=state, database_='neo4j'
        )

    else:

        neo_driver.execute_query(
            'MERGE (m: Member {head: $member}) '
            'MERGE (s: State {head: $state}) '
            'MERGE (d: District {head: $district}) '
            'MERGE (m)-[:REPRESENTS]->(d)' 
            'MERGE (d)-[:TO]->(s)',
            member=member, state=state, district=district, database_='neo4j'
        )

def add_congress_member(member, congress):

    neo_driver.execute_query(
        'MERGE (m: Member {head: $member}) '
        'MERGE (c: Congress {head: $congress}) '
        'MERGE (m)-[:MEMBER_OF]->(c)',
        member=member, congress=congress, database_='neo4j'
    )

def add_committee_member(member, committee):

    neo_driver.execute_query(
        'MERGE (m: Member {head: $member}) '
        'MERGE (k: Committee {head: $committee}) '
        'MERGE (m)-[:MEMBER_OF]->(k)',
        member=member, committee=committee, database_='neo4j'
    )

def add_sponsor(member, bill):

    neo_driver.execute_query(
        'MERGE (m: Member {head: $member}) '
        'MERGE (b: Bill {head: $bill}) '
        'MERGE (m)-[:SPONSOR]->(b)',
        member=member, bill=bill, database_='neo4j'
    )

def add_cosponsor(member, bill):

    neo_driver.execute_query(
        'MERGE (m: Member {head: $member}) '
        'MERGE (b: Bill {head: $bill}) '
        'MERGE (m)-[:COSPONSOR]->(b)',
        member=member, bill=bill, database_='neo4j'
    )

def add_yay_vote(member, bill):

    neo_driver.execute_query(
        'MERGE (m: Member {head: $member}) '
        'MERGE (b: Bill {head: $bill}) '
        'MERGE (m)-[:YAY_VOTE]->(b)',
        member=member, bill=bill, database_='neo4j'
    )

def add_nay_vote(member, bill):

    neo_driver.execute_query(
        'MERGE (m: Member {head: $member}) '
        'MERGE (b: Bill {head: $bill}) '
        'MERGE (m)-[:NAY_VOTE]->(b)',
        member=member, bill=bill, database_='neo4j'
    )

In [32]:
r = requests.get(os.getenv('CONGRESS_URL') + 'member/', params=payload)

In [33]:
count = 0

for m in r.json()['members']:
    
    if (count == 100):
        break

    member_r = requests.get(os.getenv('CONGRESS_URL') + 'member/' + m['bioguideId'], params=payload)
        
    result = member_r.json()['member']
    
    member = result['directOrderName']
    
    for party in result['partyHistory']:
        
        add_party_member(member, party['partyName'])
        
    for term in result['terms']:
        
        add_congress_member(member, term['chamber'] + ' ' + str(term['congress']))
        
        if term.get('district') is None:
            add_representation(member, term['stateName'])
        else:
            add_representation(member, term['stateName'], term['district'])
            
    count += 1